In [13]:
import pandas as pd

In [14]:
df=pd.read_excel("Churn_Management.xlsx")

In [15]:
def check_df(dataframe):
    print(f"""
        ##################### Shape #####################\n\n\t{dataframe.shape}\n\n
        ##################### Types #####################\n\n{dataframe.dtypes}\n\n
        ##################### Head #####################\n\n{dataframe.head(5)}\n\n
        ##################### Tail #####################\n\n{dataframe.tail(5)}\n\n
        ##################### NA #####################\n\n{dataframe.isnull().sum()}\n\n
        """)


In [16]:
check_df(df)


        ##################### Shape #####################

	(7043, 21)


        ##################### Types #####################

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object


        ##################### Head #####################

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No 

In [17]:
df["SeniorCitizen"] = df["SeniorCitizen"].apply(lambda x: "J" if x == 0 else "S")

In [18]:
df["tenureSegment"]=pd.cut(df["tenure"],bins=[0,6,16,float("inf")],labels=["0-6","7-16","17+"],right=True,include_lowest=True)

In [19]:
df["Contract"]=df["Contract"].map({"Month-to-month":"MM","One year":"OY","Two year":"TY"})

In [20]:
df["useTechProduct"] = df[["OnlineSecurity", "OnlineBackup", "DeviceProtection","TechSupport"]].apply(lambda x: "use_tech_product" if(x == "Yes").any() else "not_use_tech_product", axis=1)


In [21]:
df["customerSegment"]=df["SeniorCitizen"].astype(str) + " " +df["tenureSegment"].astype(str) + " " +df["Contract"].astype(str) + " " +df["useTechProduct"].astype(str)

In [22]:
df["intChurn"]=df["Churn"].map({"Yes": 1,"No": 0})

In [23]:
churn_rate=(df["Churn"].value_counts()["Yes"]/df.shape[0])*100 # result = 26.53


In [25]:
df.groupby("customerSegment",as_index=False).agg({"intChurn":"mean","customerID":"count","MonthlyCharges":"mean","tenure":"mean"}).sort_values(by="intChurn",ascending=False).head(15)

,customerSegment,intChurn,customerID,MonthlyCharges,tenure
18,S 0-6 MM not_use_tech_product,0.746479,142,64.847535,2.246479
19,S 0-6 MM use_tech_product,0.693333,75,73.504000,3.066667
26,S 7-16 MM not_use_tech_product,0.626667,75,76.569333,11.200000
27,S 7-16 MM use_tech_product,0.579545,88,78.400568,11.125000
1,J 0-6 MM use_tech_product,0.522673,419,66.924702,3.014320
0,J 0-6 MM not_use_tech_product,0.518662,777,46.578893,2.181467
20,S 17+ MM not_use_tech_product,0.489796,98,78.267347,33.051020
21,S 17+ MM use_tech_product,0.416413,329,87.000912,39.811550
13,J 7-16 MM use_tech_product,0.413514,370,72.911081,11.283784
3,J 0-6 OY use_tech_product,0.400000,5,73.790000,3.800000
